# Problem set 8
### Question 0
Here is the script we use to run in the Great Lakes.
We only change a little bit to the import part so that
it imports all the module we will use in the whole Problem Set
while the the script we actually run does not have module it does not need.
Here we use two cells to import the module we need is because
if we import tensorflow or sklearn, then we can not run pyspark in jupyter.
Hence, we adjust the import before we run qs2.
(So the second cell does not have run number here)

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean
import findspark

In [ ]:
from keras import layers,models,losses,optimizers,regularizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import multiprocess as mp
from sklearn.ensemble import GradientBoostingRegressor
import time
from sklearn.metrics import mean_squared_error

Here we revise ```cv_fold```and ```mse_score```(```gb_score```)
function in ```cv_funcs``` a little bit to fit our data.

In [2]:
def cv_fold(fit, data, idx_train, idx_val, score, label=None):
    """
    Compute score for a cross-validation fold.

    Parameters
    ----------
    fit - callable.
    A callable to fit an estimator to the training data.
    data - dataframe
    Features and labels for training and validation folds.
    idx_train, idx_val - ndarray or slice.
    Indices for subsetting x and y into training and validation folds.
    More accurately, it should be the index of dataframe instead of raw number.
    score - callable
    Function with signature (res, x, y) for scoring validation sample
    predictions from the estimator fit to training data.
    label - string or None, optional.
    An optional label for tracking results during parallel execution.
    The default is None.

    Returns
    -------
    A tuple (label, metrics) where metrics is the object returned by the
    function passed to score.

    """
    # fit model
    res = fit(data.loc[idx_train].values[:,:-1], data.loc[idx_train].values[:,-1])

    # compute score(s)
    metrics = score(res, data.loc[idx_val].values[:,:-1], data.loc[idx_val].values[:,-1])

    # return scores and label
    return ((label, metrics))

For rest functions in ```cv_funcs```, we remain the same.

In [3]:
def mse_score(res, x_v, y_v):
    """
    Compute cross entropy and accuracy at each stage of boosting model.

    Parameters
    ----------
    res = an object returned by sklearn.GradientBoosting*
    It's fit method should be called. Th object's predict()
    method is called.

    x_v, y_v - Validation features and labels

    Returns
    -------
    A dictionary with entries `Mse` for the validation
    cross mse after each fold.
    """

    # accuracy
    y_hat = res.predict(x_v)
    mse = np.mean((y_hat-y_v)**2)

    return ({ "Mse": mse})

In [4]:
# multiprocess helper functions: ----------------------------------------------
def calculate(func, args):
    """
    Call func(*args) as part of a worker process.

    Parameters
    ----------
    func - callable function
    args - a tuple of positional args for func

    Returns
    -------
    The result of `func(*args)`

    """
    result = func(*args)
    return (result)

In [5]:
def worker(input, output):
    """
    Function run by worker processes.

    Taken from
    <https://docs.python.org/3/library/multiprocessing.html
    #multiprocessing-programming>

    Parameters
    ----------
    input, output - Queues.
    Input and output queues.

    Returns
    -------
    None, called for it's side effects.
    """
    for func, args in iter(input.get, 'STOP'):
        result = calculate(func, args)
        output.put(result)

In [6]:
def mp_apply(tasks, n_processes=2):
    """
    Compute tasks in parallel.

    Parameters
    ----------
    tasks - list of tuples.
    A list of tasks, each formulated as a tuple with first element a callable
    and second a tuple of positional arguments.
    n_processes - int, optional.
    The number of child processes used to compute the tasks. The default is 2.

    Returns
    -------
    The unordered results of the computed tasks.

    """
    # create queues
    task_queue = mp.Queue()
    done_queue = mp.Queue()

    # submit tasks
    for task in tasks:
        task_queue.put(task)

    # start processes
    for i in range(n_processes):
        mp.Process(target=worker, args=(task_queue, done_queue)).start()

    # get unordered results
    results = []
    for i, task in enumerate(tasks):
        results.append(done_queue.get())

    # stop child processes
    for i in range(n_processes):
        task_queue.put('STOP')

    return (results)

First, we prepare all the data, And get the whole data set of training data.

In [7]:
np.random.seed(42)
data = pd.read_csv("train.csv")
mater = pd.read_csv("unique_m.csv")
unique_mat = mater['material'].unique()
np.random.shuffle(unique_mat)
train_mat = unique_mat[:int(len(unique_mat)*0.8)]
val_mat = unique_mat[int(len(unique_mat)*0.8):int(len(unique_mat)*0.9)]
test_mat = unique_mat[int(len(unique_mat)*0.9):]
train_index = mater[mater['material'].isin(train_mat)].index.values
val_index = mater[mater['material'].isin(val_mat)].index.values
test_index = mater[mater['material'].isin(test_mat)].index.values

In [8]:
train_data = data.loc[train_index]
val_data = data.loc[val_index]
test_data = data.loc[test_index]
x_val = val_data.values[:,:-1]
y_val = val_data.values[:,-1]
x_test = test_data.values[:,:-1]
y_test = test_data.values[:,-1]
x_train = train_data.values[:,:-1]
y_train = train_data.values[:,-1]

Next, we prepare the index of 10-fold training set and testing set in the whole traning data.

In [9]:
fold_mat = []
for i in range(9):
    fold_test_mat = train_mat[i*1243:(i+1)*1243]
    fold_train_mat = list(set(train_mat)-set(fold_test_mat))
    fold_mat.append((fold_train_mat,fold_test_mat))
fold_mat.append((train_mat[:9*1243],train_mat[9*1243:]))
fold_index = []
for fold in fold_mat:
    fold_train_index = mater[mater['material'].isin(fold[0])].index.values
    fold_test_index = mater[mater['material'].isin(fold[1])].index.values
    fold_index.append((fold_train_index,fold_test_index))

In [10]:
gdbt = GradientBoostingRegressor(learning_rate=0.25,random_state=2021,n_estimators=390)
n_processes = 5

Prepare the all the tasks.

In [11]:
cv_tasks = []
for fold, (idx_train, idx_val) in enumerate(fold_index):
    task = (cv_fold,
            (gdbt.fit,
             train_data,
             idx_train,
             idx_val,
             mse_score,
             'gbdt_fold' + str(fold)
            )
           )
    cv_tasks.append(task)

We find that jupyter notebook cannot run mulitprocessing
unless we put our function into other files.
Hence we will not run the following code
here to make what we upload concisely.
And it can run in Pycharm or in Great Lakes.

In [ ]:
if __name__ == "__main__":
    print('Start: ' + time.ctime())
    result0 = mp_apply(cv_tasks,n_processes)
    print('End: ' + time.ctime())
    avg_mse = 0
    for result in result0:
        avg_mse += result[1]['Mse']/10
    print("10 folds mse:",avg_mse)
    model_res = gdbt.fit(x_train,y_train)
    y_hat_val = model_res.predict(x_val)
    y_hat_test = model_res.predict(x_test)
    mse_val = mean_squared_error(y_hat_val,y_val)
    mse_test = mean_squared_error(y_hat_test,y_test)
    print("validation mse:{:.3f},test mse:{:.3f}".format(mse_val,mse_test))

Start: Fri Dec  3 19:00:29 2021


The following code is sh script we use

\#!/usr/bin/bash

\#

\# Author: Tiejin Chen

\# Updated: Dec 03, 2021

\# slurm options: --------------------------------------------------------------

#SBATCH --job-name=tiejin_ps8qs0

#SBATCH --mail-user=tiejin@umich.edu

#SBATCH --mail-type=BEGIN,END

#SBATCH --cpus-per-task=5

#SBATCH --nodes=1

#SBATCH --ntasks-per-node=1

#SBATCH --mem-per-cpu=5GB

#SBATCH --time=10:00

#SBATCH --account=stats507f21_class

#SBATCH --partition=standard

#SBATCH --output=/home/%u/logs/%x-%j-5.log

\# application: ----------------------------------------------------------------

\# modules 

#SBATCH --get-user-env

\# the contents of this script

cat run-tiejin_qs0.sh

\# run the script

date

cd /home/tiejin/

python PS8_chentiejin_qs0.py

date

echo "Done."

And the usage information we get from the command line is:


| JobID | NCPUS  |  Elapsed | CPUTime | TotalCPU |
|  ----  | ---- | ---- | ---- | ---- |
| 29528720 | 5 |   00:04:19 |  00:21:35 |  15:12.804 |
| 29528720.ba+ | 5 |   00:04:19 |  00:21:35 |  15:12.803 |
| 29528720.ex+ | 5 |   00:04:19 |  00:21:35 | 00:00:00 |


We can also get the log:

```#!/usr/bin/bash
#
# Author: Tiejin Chen
# Updated: Dec 03, 2021
# 1: -------------------------------------------------------------------------

# slurm options: --------------------------------------------------------------
#SBATCH --job-name=tiejin_ps8qs0
#SBATCH --mail-user=tiejin@umich.edu
#SBATCH --mail-type=BEGIN,END
#SBATCH --cpus-per-task=5
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --mem-per-cpu=5GB
#SBATCH --time=10:00
#SBATCH --account=stats507f21_class
#SBATCH --partition=standard
#SBATCH --output=/home/%u/logs/%x-%j-5.log

# application: ----------------------------------------------------------------

# modules 
#SBATCH --get-user-env

# the contents of this script
cat run-tiejin_qs0.sh

# run the script
date

cd /home/tiejin/
python PS8_chentiejin_qs0.py

date
echo "Done."

Fri Dec  3 17:15:13 EST 2021
Start: Fri Dec  3 17:15:15 2021
End: Fri Dec  3 17:17:59 2021
10 folds mse: 117.09980684701303
validation mse:109.446,test mse:112.227
Fri Dec  3 17:19:30 EST 2021
Done.


### Question 1
#### Part a
in this part, we will consider 6 models:
1. model with 3 hidden layers
2. model with 1 hidden layer
3. model with 2 hidden layers
4. model with 3 hidden layers and l2 norm
5. model with 3 hidden layers and l1 norm
6. model with 3 hidden layers and dropout

All the models are trained with ```mse``` loss and we will use ```relu```
as the activation function. All the model will be optimized by Adam with learning rate is 1e-4.
We set batch_size to 32 and epochs to 500 for every model.
For the parameter of normliztion, we will set it to 0.005 for all models.
And we will set dropout fraction to 0.3.
Also we only save the best model during 500 epochs
with least validation mse, and using the least validation mse to get the best model.

According to the requirement,
we are not required to search best hyper-parameter here
and the way we choose the best model is to see their performence in validation set.
There is no reason to do the cross-validation here because
in the last we will use the whole data to re-train the model
and get their last model performance in validation set. Cross-validation
does not do anything helpful in this process. In contrast,
If we use cross-validation, we will get 10 models in one kind of model.
And comparing model_list's performance is quite a large and open question.
Overall, we will directly use whole train data to fit 6 models.


In [12]:
model1 = models.Sequential()
model1.add(layers.Dense(256,activation="relu",input_dim=81))
model1.add(layers.Dense(128,activation="relu"))
model1.add(layers.Dense(128,activation="relu"))
model1.add(layers.Dense(1))
model1.compile(
    optimizers.Adam(learning_rate=1e-4),
    losses.mean_squared_error,
    metrics = [losses.mean_squared_error]
)
file_path = "model1.hdf5"
checkpoint = ModelCheckpoint(file_path,monitor='val_loss', verbose=1,
                             save_best_only=True,period=1)
train_history1 = model1.fit(x_train,y_train,epochs=500,batch_size = 32,
                            validation_data = (x_val,y_val),callbacks=[checkpoint])

Epoch 1/500
510/526 [============================>.] - ETA: 0s - loss: 3251.3567 - mean_squared_error: 3251.3567
Epoch 00001: val_loss improved from inf to 618.03674, saving model to model1.hdf5
526/526 [==============================] - 1s 1ms/step - loss: 3172.5859 - mean_squared_error: 3172.5859 - val_loss: 618.0367 - val_mean_squared_error: 618.0367
Epoch 2/500
438/526 [=======================>......] - ETA: 0s - loss: 522.0392 - mean_squared_error: 522.0392
Epoch 00002: val_loss improved from 618.03674 to 477.32318, saving model to model1.hdf5
526/526 [==============================] - 0s 786us/step - loss: 512.6707 - mean_squared_error: 512.6707 - val_loss: 477.3232 - val_mean_squared_error: 477.3232
Epoch 3/500
436/526 [=======================>......] - ETA: 0s - loss: 456.0528 - mean_squared_error: 456.0528
Epoch 00003: val_loss did not improve from 477.32318
526/526 [==============================] - 0s 510us/step - loss: 452.2300 - mean_squared_error: 452.2300 - val_loss: 548

Epoch 25/500
520/526 [============================>.] - ETA: 0s - loss: 312.3293 - mean_squared_error: 312.3293
Epoch 00025: val_loss improved from 279.65335 to 273.08157, saving model to model1.hdf5
526/526 [==============================] - 0s 788us/step - loss: 311.3496 - mean_squared_error: 311.3496 - val_loss: 273.0816 - val_mean_squared_error: 273.0816
Epoch 26/500
522/526 [============================>.] - ETA: 0s - loss: 284.8234 - mean_squared_error: 284.8234
Epoch 00026: val_loss did not improve from 273.08157
526/526 [==============================] - 0s 531us/step - loss: 285.0126 - mean_squared_error: 285.0126 - val_loss: 312.0334 - val_mean_squared_error: 312.0334
Epoch 27/500
523/526 [============================>.] - ETA: 0s - loss: 290.3086 - mean_squared_error: 290.3086
Epoch 00027: val_loss did not improve from 273.08157
526/526 [==============================] - 0s 529us/step - loss: 290.4068 - mean_squared_error: 290.4068 - val_loss: 312.2248 - val_mean_squared_err

Epoch 50/500
428/526 [=======================>......] - ETA: 0s - loss: 260.9003 - mean_squared_error: 260.9003
Epoch 00050: val_loss did not improve from 241.11096
526/526 [==============================] - 0s 516us/step - loss: 260.0759 - mean_squared_error: 260.0759 - val_loss: 444.4107 - val_mean_squared_error: 444.4107
Epoch 51/500
431/526 [=======================>......] - ETA: 0s - loss: 261.8663 - mean_squared_error: 261.8663
Epoch 00051: val_loss did not improve from 241.11096
526/526 [==============================] - 0s 512us/step - loss: 258.5799 - mean_squared_error: 258.5799 - val_loss: 323.3438 - val_mean_squared_error: 323.3438
Epoch 52/500
428/526 [=======================>......] - ETA: 0s - loss: 238.5322 - mean_squared_error: 238.5322
Epoch 00052: val_loss improved from 241.11096 to 237.50722, saving model to model1.hdf5
526/526 [==============================] - 0s 712us/step - loss: 242.8317 - mean_squared_error: 242.8317 - val_loss: 237.5072 - val_mean_squared_err

Epoch 75/500
429/526 [=======================>......] - ETA: 0s - loss: 218.1154 - mean_squared_error: 218.1154
Epoch 00075: val_loss did not improve from 219.28242
526/526 [==============================] - 0s 519us/step - loss: 219.1428 - mean_squared_error: 219.1428 - val_loss: 241.1568 - val_mean_squared_error: 241.1568
Epoch 76/500
430/526 [=======================>......] - ETA: 0s - loss: 221.2676 - mean_squared_error: 221.2676
Epoch 00076: val_loss improved from 219.28242 to 215.64386, saving model to model1.hdf5
526/526 [==============================] - 0s 769us/step - loss: 224.0114 - mean_squared_error: 224.0114 - val_loss: 215.6439 - val_mean_squared_error: 215.6439
Epoch 77/500
433/526 [=======================>......] - ETA: 0s - loss: 213.3620 - mean_squared_error: 213.3620
Epoch 00077: val_loss did not improve from 215.64386
526/526 [==============================] - 0s 510us/step - loss: 217.7363 - mean_squared_error: 217.7363 - val_loss: 215.7356 - val_mean_squared_err

Epoch 100/500
525/526 [============================>.] - ETA: 0s - loss: 201.5186 - mean_squared_error: 201.5186
Epoch 00100: val_loss did not improve from 194.10236
526/526 [==============================] - 0s 527us/step - loss: 201.2771 - mean_squared_error: 201.2771 - val_loss: 221.7829 - val_mean_squared_error: 221.7829
Epoch 101/500
429/526 [=======================>......] - ETA: 0s - loss: 202.1317 - mean_squared_error: 202.1317
Epoch 00101: val_loss did not improve from 194.10236
526/526 [==============================] - 0s 516us/step - loss: 204.3895 - mean_squared_error: 204.3895 - val_loss: 195.9960 - val_mean_squared_error: 195.9960
Epoch 102/500
500/526 [===========================>..] - ETA: 0s - loss: 204.4950 - mean_squared_error: 204.4950
Epoch 00102: val_loss did not improve from 194.10236
526/526 [==============================] - 0s 556us/step - loss: 204.2354 - mean_squared_error: 204.2354 - val_loss: 209.8718 - val_mean_squared_error: 209.8718
Epoch 103/500
490/5

...

428/526 [=======================>......] - ETA: 0s - loss: 111.0095 - mean_squared_error: 111.0095
Epoch 00375: val_loss did not improve from 129.34189
526/526 [==============================] - 0s 516us/step - loss: 110.7927 - mean_squared_error: 110.7927 - val_loss: 143.8042 - val_mean_squared_error: 143.8042
Epoch 376/500
424/526 [=======================>......] - ETA: 0s - loss: 111.0882 - mean_squared_error: 111.0882
Epoch 00376: val_loss did not improve from 129.34189
526/526 [==============================] - 0s 525us/step - loss: 112.1952 - mean_squared_error: 112.1952 - val_loss: 131.9046 - val_mean_squared_error: 131.9046
Epoch 377/500
428/526 [=======================>......] - ETA: 0s - loss: 111.7324 - mean_squared_error: 111.7324
Epoch 00377: val_loss did not improve from 129.34189
526/526 [==============================] - 0s 516us/step - loss: 111.7837 - mean_squared_error: 111.7837 - val_loss: 135.8340 - val_mean_squared_error: 135.8340
Epoch 378/500
424/526 [==========

Epoch 400/500
428/526 [=======================>......] - ETA: 0s - loss: 108.0156 - mean_squared_error: 108.0156
Epoch 00400: val_loss did not improve from 128.77081
526/526 [==============================] - 0s 516us/step - loss: 110.6158 - mean_squared_error: 110.6158 - val_loss: 147.3625 - val_mean_squared_error: 147.3625
Epoch 401/500
424/526 [=======================>......] - ETA: 0s - loss: 112.2290 - mean_squared_error: 112.2290
Epoch 00401: val_loss did not improve from 128.77081
526/526 [==============================] - 0s 519us/step - loss: 110.5598 - mean_squared_error: 110.5598 - val_loss: 147.9990 - val_mean_squared_error: 147.9990
Epoch 402/500
432/526 [=======================>......] - ETA: 0s - loss: 110.3440 - mean_squared_error: 110.3440
Epoch 00402: val_loss did not improve from 128.77081
526/526 [==============================] - 0s 512us/step - loss: 111.5364 - mean_squared_error: 111.5364 - val_loss: 144.6127 - val_mean_squared_error: 144.6127
Epoch 403/500
427/5

Epoch 425/500
426/526 [=======================>......] - ETA: 0s - loss: 104.4922 - mean_squared_error: 104.4922
Epoch 00425: val_loss did not improve from 124.61942
526/526 [==============================] - 0s 519us/step - loss: 105.3479 - mean_squared_error: 105.3479 - val_loss: 126.0041 - val_mean_squared_error: 126.0041
Epoch 426/500
424/526 [=======================>......] - ETA: 0s - loss: 104.4612 - mean_squared_error: 104.4612
Epoch 00426: val_loss improved from 124.61942 to 123.40611, saving model to model1.hdf5
526/526 [==============================] - 0s 666us/step - loss: 104.4666 - mean_squared_error: 104.4666 - val_loss: 123.4061 - val_mean_squared_error: 123.4061
Epoch 427/500
425/526 [=======================>......] - ETA: 0s - loss: 106.0058 - mean_squared_error: 106.0058
Epoch 00427: val_loss did not improve from 123.40611
526/526 [==============================] - 0s 519us/step - loss: 105.9179 - mean_squared_error: 105.9179 - val_loss: 152.7446 - val_mean_squared_

Epoch 450/500
427/526 [=======================>......] - ETA: 0s - loss: 102.9436 - mean_squared_error: 102.9436
Epoch 00450: val_loss did not improve from 123.40611
526/526 [==============================] - 0s 519us/step - loss: 102.4376 - mean_squared_error: 102.4376 - val_loss: 131.9510 - val_mean_squared_error: 131.9510
Epoch 451/500
526/526 [==============================] - ETA: 0s - loss: 104.3370 - mean_squared_error: 104.3370
Epoch 00451: val_loss did not improve from 123.40611
526/526 [==============================] - 0s 525us/step - loss: 104.3370 - mean_squared_error: 104.3370 - val_loss: 133.8362 - val_mean_squared_error: 133.8362
Epoch 452/500
421/526 [=======================>......] - ETA: 0s - loss: 103.9166 - mean_squared_error: 103.9166
Epoch 00452: val_loss did not improve from 123.40611
526/526 [==============================] - 0s 523us/step - loss: 102.8290 - mean_squared_error: 102.8290 - val_loss: 132.7946 - val_mean_squared_error: 132.7946
Epoch 453/500
424/5

421/526 [=======================>......] - ETA: 0s - loss: 98.5449 - mean_squared_error: 98.5449
Epoch 00500: val_loss did not improve from 123.40611
526/526 [==============================] - 0s 525us/step - loss: 99.2952 - mean_squared_error: 99.2952 - val_loss: 140.1748 - val_mean_squared_error: 140.1748


In [13]:
model2 = models.Sequential()
model2.add(layers.Dense(256,activation="relu",input_dim=81))
model2.add(layers.Dense(1))
model2.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss = losses.mean_squared_error,
    metrics = ['mean_squared_error']
)
file_path = "model2.hdf5"
checkpoint = ModelCheckpoint(file_path,monitor='val_loss', verbose=1,
                             save_best_only=True,period=1)
train_history2 = model2.fit(x_train,y_train,epochs=500,batch_size = 32,
                            validation_data = (x_val,y_val),callbacks = [checkpoint])

Epoch 1/500
444/526 [========================>.....] - ETA: 0s - loss: 38190.9688 - mean_squared_error: 38190.9688
Epoch 00001: val_loss improved from inf to 3169.97461, saving model to model2.hdf5
526/526 [==============================] - 0s 658us/step - loss: 32837.5000 - mean_squared_error: 32837.5000 - val_loss: 3169.9746 - val_mean_squared_error: 3169.9746
Epoch 2/500
451/526 [========================>.....] - ETA: 0s - loss: 2291.8909 - mean_squared_error: 2291.8909
Epoch 00002: val_loss improved from 3169.97461 to 1573.40198, saving model to model2.hdf5
526/526 [==============================] - 0s 502us/step - loss: 2192.9316 - mean_squared_error: 2192.9316 - val_loss: 1573.4020 - val_mean_squared_error: 1573.4020
Epoch 3/500
451/526 [========================>.....] - ETA: 0s - loss: 1273.4939 - mean_squared_error: 1273.4939
Epoch 00003: val_loss improved from 1573.40198 to 1052.36792, saving model to model2.hdf5
526/526 [==============================] - 0s 489us/step - loss:

Epoch 25/500
450/526 [========================>.....] - ETA: 0s - loss: 364.2608 - mean_squared_error: 364.2608
Epoch 00025: val_loss improved from 304.73474 to 295.90784, saving model to model2.hdf5
526/526 [==============================] - 0s 526us/step - loss: 363.5198 - mean_squared_error: 363.5198 - val_loss: 295.9078 - val_mean_squared_error: 295.9078
Epoch 26/500
447/526 [========================>.....] - ETA: 0s - loss: 340.2424 - mean_squared_error: 340.2424
Epoch 00026: val_loss improved from 295.90784 to 283.16986, saving model to model2.hdf5
526/526 [==============================] - 0s 415us/step - loss: 338.2516 - mean_squared_error: 338.2516 - val_loss: 283.1699 - val_mean_squared_error: 283.1699
Epoch 27/500
440/526 [========================>.....] - ETA: 0s - loss: 343.7614 - mean_squared_error: 343.7614
Epoch 00027: val_loss did not improve from 283.16986
526/526 [==============================] - 0s 386us/step - loss: 339.3099 - mean_squared_error: 339.3099 - val_lo

Epoch 50/500
446/526 [========================>.....] - ETA: 0s - loss: 286.2586 - mean_squared_error: 286.2586
Epoch 00050: val_loss did not improve from 273.61508
526/526 [==============================] - 0s 381us/step - loss: 292.1905 - mean_squared_error: 292.1905 - val_loss: 280.6726 - val_mean_squared_error: 280.6726
Epoch 51/500
439/526 [========================>.....] - ETA: 0s - loss: 306.8182 - mean_squared_error: 306.8182
Epoch 00051: val_loss did not improve from 273.61508
526/526 [==============================] - 0s 384us/step - loss: 304.7008 - mean_squared_error: 304.7008 - val_loss: 302.0627 - val_mean_squared_error: 302.0627
Epoch 52/500
452/526 [========================>.....] - ETA: 0s - loss: 300.4867 - mean_squared_error: 300.4867
Epoch 00052: val_loss improved from 273.61508 to 263.26141, saving model to model2.hdf5
526/526 [==============================] - 0s 481us/step - loss: 297.5890 - mean_squared_error: 297.5890 - val_loss: 263.2614 - val_mean_squared_err

Epoch 75/500
411/526 [======================>.......] - ETA: 0s - loss: 293.5168 - mean_squared_error: 293.5168
Epoch 00075: val_loss did not improve from 257.16266
526/526 [==============================] - 0s 407us/step - loss: 294.0330 - mean_squared_error: 294.0330 - val_loss: 395.6811 - val_mean_squared_error: 395.6811
Epoch 76/500
432/526 [=======================>......] - ETA: 0s - loss: 275.6886 - mean_squared_error: 275.6886
Epoch 00076: val_loss did not improve from 257.16266
526/526 [==============================] - 0s 392us/step - loss: 280.8712 - mean_squared_error: 280.8712 - val_loss: 668.3412 - val_mean_squared_error: 668.3412
Epoch 77/500
441/526 [========================>.....] - ETA: 0s - loss: 296.9796 - mean_squared_error: 296.9796
Epoch 00077: val_loss did not improve from 257.16266
526/526 [==============================] - 0s 388us/step - loss: 290.9227 - mean_squared_error: 290.9227 - val_loss: 310.2321 - val_mean_squared_error: 310.2321
Epoch 78/500
439/526 [

Epoch 100/500
445/526 [========================>.....] - ETA: 0s - loss: 272.4223 - mean_squared_error: 272.4223
Epoch 00100: val_loss did not improve from 241.97948
526/526 [==============================] - 0s 382us/step - loss: 272.2757 - mean_squared_error: 272.2757 - val_loss: 250.2926 - val_mean_squared_error: 250.2926
Epoch 101/500
454/526 [========================>.....] - ETA: 0s - loss: 287.4725 - mean_squared_error: 287.4725
Epoch 00101: val_loss did not improve from 241.97948
526/526 [==============================] - 0s 373us/step - loss: 286.6204 - mean_squared_error: 286.6204 - val_loss: 279.8168 - val_mean_squared_error: 279.8168
Epoch 102/500
462/526 [=========================>....] - ETA: 0s - loss: 264.3965 - mean_squared_error: 264.3965
Epoch 00102: val_loss did not improve from 241.97948
526/526 [==============================] - 0s 369us/step - loss: 266.3490 - mean_squared_error: 266.3490 - val_loss: 263.7960 - val_mean_squared_error: 263.7960
Epoch 103/500
448/5

...

Epoch 275/500
443/526 [========================>.....] - ETA: 0s - loss: 238.9635 - mean_squared_error: 238.9635
Epoch 00275: val_loss did not improve from 214.70380
526/526 [==============================] - 0s 384us/step - loss: 239.0907 - mean_squared_error: 239.0907 - val_loss: 216.8045 - val_mean_squared_error: 216.8045
Epoch 276/500
448/526 [========================>.....] - ETA: 0s - loss: 232.5249 - mean_squared_error: 232.5249
Epoch 00276: val_loss did not improve from 214.70380
526/526 [==============================] - 0s 379us/step - loss: 233.9292 - mean_squared_error: 233.9292 - val_loss: 506.6071 - val_mean_squared_error: 506.6071
Epoch 277/500
431/526 [=======================>......] - ETA: 0s - loss: 245.2328 - mean_squared_error: 245.2328
Epoch 00277: val_loss did not improve from 214.70380
526/526 [==============================] - 0s 392us/step - loss: 248.8752 - mean_squared_error: 248.8752 - val_loss: 251.1435 - val_mean_squared_error: 251.1435
Epoch 278/500
445/5

Epoch 325/500
450/526 [========================>.....] - ETA: 0s - loss: 234.1183 - mean_squared_error: 234.1183
Epoch 00325: val_loss did not improve from 212.93303
526/526 [==============================] - 0s 379us/step - loss: 236.0359 - mean_squared_error: 236.0359 - val_loss: 321.4800 - val_mean_squared_error: 321.4800
Epoch 326/500
458/526 [=========================>....] - ETA: 0s - loss: 236.5617 - mean_squared_error: 236.5617
Epoch 00326: val_loss did not improve from 212.93303
526/526 [==============================] - 0s 373us/step - loss: 232.2254 - mean_squared_error: 232.2254 - val_loss: 225.9644 - val_mean_squared_error: 225.9644
Epoch 327/500
453/526 [========================>.....] - ETA: 0s - loss: 242.5057 - mean_squared_error: 242.5057
Epoch 00327: val_loss did not improve from 212.93303
526/526 [==============================] - 0s 377us/step - loss: 241.1563 - mean_squared_error: 241.1563 - val_loss: 222.2546 - val_mean_squared_error: 222.2546
Epoch 328/500
459/5

444/526 [========================>.....] - ETA: 0s - loss: 232.9761 - mean_squared_error: 232.9761
Epoch 00375: val_loss did not improve from 212.93303
526/526 [==============================] - 0s 382us/step - loss: 232.9666 - mean_squared_error: 232.9666 - val_loss: 276.2115 - val_mean_squared_error: 276.2115
Epoch 376/500
438/526 [=======================>......] - ETA: 0s - loss: 225.6826 - mean_squared_error: 225.6826
Epoch 00376: val_loss did not improve from 212.93303
526/526 [==============================] - 0s 388us/step - loss: 227.4521 - mean_squared_error: 227.4521 - val_loss: 224.3303 - val_mean_squared_error: 224.3303
Epoch 377/500
446/526 [========================>.....] - ETA: 0s - loss: 248.3457 - mean_squared_error: 248.3457
Epoch 00377: val_loss did not improve from 212.93303
526/526 [==============================] - 0s 381us/step - loss: 242.7788 - mean_squared_error: 242.7788 - val_loss: 230.3757 - val_mean_squared_error: 230.3757
Epoch 378/500
453/526 [==========

Epoch 425/500
445/526 [========================>.....] - ETA: 0s - loss: 225.6297 - mean_squared_error: 225.6297
Epoch 00425: val_loss did not improve from 211.79543
526/526 [==============================] - 0s 382us/step - loss: 223.7798 - mean_squared_error: 223.7798 - val_loss: 226.1863 - val_mean_squared_error: 226.1863
Epoch 426/500
449/526 [========================>.....] - ETA: 0s - loss: 226.3988 - mean_squared_error: 226.3988
Epoch 00426: val_loss did not improve from 211.79543
526/526 [==============================] - 0s 379us/step - loss: 224.6519 - mean_squared_error: 224.6519 - val_loss: 242.2802 - val_mean_squared_error: 242.2802
Epoch 427/500
404/526 [======================>.......] - ETA: 0s - loss: 236.9142 - mean_squared_error: 236.9142
Epoch 00427: val_loss did not improve from 211.79543
526/526 [==============================] - 0s 413us/step - loss: 237.8291 - mean_squared_error: 237.8291 - val_loss: 230.5564 - val_mean_squared_error: 230.5564
Epoch 428/500
435/5

450/526 [========================>.....] - ETA: 0s - loss: 220.9868 - mean_squared_error: 220.9868
Epoch 00475: val_loss did not improve from 211.79543
526/526 [==============================] - 0s 379us/step - loss: 223.1409 - mean_squared_error: 223.1409 - val_loss: 228.6304 - val_mean_squared_error: 228.6304
Epoch 476/500
446/526 [========================>.....] - ETA: 0s - loss: 215.5300 - mean_squared_error: 215.5300
Epoch 00476: val_loss did not improve from 211.79543
526/526 [==============================] - 0s 382us/step - loss: 216.5407 - mean_squared_error: 216.5407 - val_loss: 216.9460 - val_mean_squared_error: 216.9460
Epoch 477/500
440/526 [========================>.....] - ETA: 0s - loss: 220.4879 - mean_squared_error: 220.4879
Epoch 00477: val_loss did not improve from 211.79543
526/526 [==============================] - 0s 392us/step - loss: 220.4182 - mean_squared_error: 220.4182 - val_loss: 290.6028 - val_mean_squared_error: 290.6028
Epoch 478/500
439/526 [==========

In [14]:
model3 = models.Sequential()
model3.add(layers.Dense(256,activation="relu",input_dim=81))
model3.add(layers.Dense(128,activation="relu"))
model3.add(layers.Dense(1))
model3.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss = losses.mean_squared_error,
    metrics = ['mean_squared_error']
)
file_path = "model3.hdf5"
checkpoint = ModelCheckpoint(file_path,monitor='val_loss', verbose=1,
                             save_best_only=True,period=1)
train_history3 = model3.fit(x_train,y_train,epochs=500,batch_size = 32,
                            validation_data = (x_val,y_val),callbacks = [checkpoint])

Epoch 1/500
466/526 [=========================>....] - ETA: 0s - loss: 13454.8262 - mean_squared_error: 13454.8262
Epoch 00001: val_loss improved from inf to 1046.02478, saving model to model3.hdf5
526/526 [==============================] - 0s 806us/step - loss: 12029.0547 - mean_squared_error: 12029.0547 - val_loss: 1046.0248 - val_mean_squared_error: 1046.0248
Epoch 2/500
470/526 [=========================>....] - ETA: 0s - loss: 722.6699 - mean_squared_error: 722.6699
Epoch 00002: val_loss improved from 1046.02478 to 787.64496, saving model to model3.hdf5
526/526 [==============================] - 0s 542us/step - loss: 709.1683 - mean_squared_error: 709.1683 - val_loss: 787.6450 - val_mean_squared_error: 787.6450
Epoch 3/500
474/526 [==========================>...] - ETA: 0s - loss: 554.8463 - mean_squared_error: 554.8463
Epoch 00003: val_loss improved from 787.64496 to 572.37836, saving model to model3.hdf5
526/526 [==============================] - 0s 546us/step - loss: 550.2642 -

471/526 [=========================>....] - ETA: 0s - loss: 318.2573 - mean_squared_error: 318.2573
Epoch 00049: val_loss improved from 261.21494 to 243.86018, saving model to model3.hdf5
526/526 [==============================] - 0s 649us/step - loss: 312.0423 - mean_squared_error: 312.0423 - val_loss: 243.8602 - val_mean_squared_error: 243.8602
Epoch 50/500
478/526 [==========================>...] - ETA: 0s - loss: 358.8419 - mean_squared_error: 358.8419
Epoch 00050: val_loss did not improve from 243.86018
526/526 [==============================] - 0s 464us/step - loss: 353.2515 - mean_squared_error: 353.2515 - val_loss: 284.7650 - val_mean_squared_error: 284.7650
Epoch 51/500
485/526 [==========================>...] - ETA: 0s - loss: 312.0789 - mean_squared_error: 312.0789
Epoch 00051: val_loss did not improve from 243.86018
526/526 [==============================] - 0s 462us/step - loss: 310.8452 - mean_squared_error: 310.8452 - val_loss: 309.8980 - val_mean_squared_error: 309.8980


Epoch 74/500
474/526 [==========================>...] - ETA: 0s - loss: 271.1498 - mean_squared_error: 271.1498
Epoch 00074: val_loss did not improve from 243.02072
526/526 [==============================] - 0s 466us/step - loss: 274.5123 - mean_squared_error: 274.5123 - val_loss: 311.2512 - val_mean_squared_error: 311.2512
Epoch 75/500
486/526 [==========================>...] - ETA: 0s - loss: 312.6165 - mean_squared_error: 312.6165
Epoch 00075: val_loss did not improve from 243.02072
526/526 [==============================] - 0s 459us/step - loss: 315.9364 - mean_squared_error: 315.9364 - val_loss: 303.2460 - val_mean_squared_error: 303.2460
Epoch 76/500
484/526 [==========================>...] - ETA: 0s - loss: 293.7810 - mean_squared_error: 293.7810
Epoch 00076: val_loss did not improve from 243.02072
526/526 [==============================] - 0s 459us/step - loss: 299.1914 - mean_squared_error: 299.1914 - val_loss: 347.2520 - val_mean_squared_error: 347.2520
Epoch 77/500
484/526 [

Epoch 99/500
484/526 [==========================>...] - ETA: 0s - loss: 268.0264 - mean_squared_error: 268.0264
Epoch 00099: val_loss did not improve from 222.62215
526/526 [==============================] - 0s 460us/step - loss: 264.8582 - mean_squared_error: 264.8582 - val_loss: 268.6448 - val_mean_squared_error: 268.6448
Epoch 100/500
481/526 [==========================>...] - ETA: 0s - loss: 261.6875 - mean_squared_error: 261.6875
Epoch 00100: val_loss did not improve from 222.62215
526/526 [==============================] - 0s 464us/step - loss: 259.8160 - mean_squared_error: 259.8160 - val_loss: 264.3177 - val_mean_squared_error: 264.3177
Epoch 101/500
481/526 [==========================>...] - ETA: 0s - loss: 273.5508 - mean_squared_error: 273.5508
Epoch 00101: val_loss did not improve from 222.62215
526/526 [==============================] - 0s 464us/step - loss: 272.3805 - mean_squared_error: 272.3805 - val_loss: 305.2293 - val_mean_squared_error: 305.2293
Epoch 102/500
475/52

480/526 [==========================>...] - ETA: 0s - loss: 238.0966 - mean_squared_error: 238.0966
Epoch 00149: val_loss did not improve from 222.62215
526/526 [==============================] - 0s 462us/step - loss: 236.0920 - mean_squared_error: 236.0920 - val_loss: 231.1578 - val_mean_squared_error: 231.1578
Epoch 150/500
485/526 [==========================>...] - ETA: 0s - loss: 241.3213 - mean_squared_error: 241.3213
Epoch 00150: val_loss did not improve from 222.62215
526/526 [==============================] - 0s 460us/step - loss: 241.0093 - mean_squared_error: 241.0093 - val_loss: 228.3736 - val_mean_squared_error: 228.3736
Epoch 151/500
482/526 [==========================>...] - ETA: 0s - loss: 235.7823 - mean_squared_error: 235.7823
Epoch 00151: val_loss did not improve from 222.62215
526/526 [==============================] - 0s 462us/step - loss: 239.3797 - mean_squared_error: 239.3797 - val_loss: 249.3077 - val_mean_squared_error: 249.3077
Epoch 152/500
482/526 [==========

...

Epoch 399/500
488/526 [==========================>...] - ETA: 0s - loss: 154.8632 - mean_squared_error: 154.8632
Epoch 00399: val_loss did not improve from 167.18558
526/526 [==============================] - 0s 457us/step - loss: 155.0676 - mean_squared_error: 155.0676 - val_loss: 181.9279 - val_mean_squared_error: 181.9279
Epoch 400/500
487/526 [==========================>...] - ETA: 0s - loss: 160.4801 - mean_squared_error: 160.4801
Epoch 00400: val_loss did not improve from 167.18558
526/526 [==============================] - 0s 459us/step - loss: 160.1287 - mean_squared_error: 160.1287 - val_loss: 173.1421 - val_mean_squared_error: 173.1421
Epoch 401/500
484/526 [==========================>...] - ETA: 0s - loss: 154.2961 - mean_squared_error: 154.2961
Epoch 00401: val_loss did not improve from 167.18558
526/526 [==============================] - 0s 460us/step - loss: 156.4953 - mean_squared_error: 156.4953 - val_loss: 227.7485 - val_mean_squared_error: 227.7485
Epoch 402/500
486/5

Epoch 424/500
484/526 [==========================>...] - ETA: 0s - loss: 152.5333 - mean_squared_error: 152.5333
Epoch 00424: val_loss did not improve from 163.44670
526/526 [==============================] - 0s 460us/step - loss: 152.4133 - mean_squared_error: 152.4133 - val_loss: 173.1885 - val_mean_squared_error: 173.1885
Epoch 425/500
495/526 [===========================>..] - ETA: 0s - loss: 152.0885 - mean_squared_error: 152.0885
Epoch 00425: val_loss did not improve from 163.44670
526/526 [==============================] - 0s 451us/step - loss: 154.6428 - mean_squared_error: 154.6428 - val_loss: 176.9368 - val_mean_squared_error: 176.9368
Epoch 426/500
487/526 [==========================>...] - ETA: 0s - loss: 149.5692 - mean_squared_error: 149.5692
Epoch 00426: val_loss did not improve from 163.44670
526/526 [==============================] - 0s 459us/step - loss: 151.1133 - mean_squared_error: 151.1133 - val_loss: 175.4492 - val_mean_squared_error: 175.4492
Epoch 427/500
486/5

Epoch 449/500
488/526 [==========================>...] - ETA: 0s - loss: 150.5638 - mean_squared_error: 150.5638
Epoch 00449: val_loss did not improve from 159.75859
526/526 [==============================] - 0s 459us/step - loss: 150.6989 - mean_squared_error: 150.6989 - val_loss: 178.5679 - val_mean_squared_error: 178.5679
Epoch 450/500
487/526 [==========================>...] - ETA: 0s - loss: 149.5574 - mean_squared_error: 149.5574
Epoch 00450: val_loss did not improve from 159.75859
526/526 [==============================] - 0s 459us/step - loss: 150.4148 - mean_squared_error: 150.4148 - val_loss: 178.6994 - val_mean_squared_error: 178.6994
Epoch 451/500
489/526 [==========================>...] - ETA: 0s - loss: 148.0537 - mean_squared_error: 148.0537
Epoch 00451: val_loss did not improve from 159.75859
526/526 [==============================] - 0s 457us/step - loss: 147.5234 - mean_squared_error: 147.5234 - val_loss: 168.7859 - val_mean_squared_error: 168.7859
Epoch 452/500
487/5

Epoch 474/500
487/526 [==========================>...] - ETA: 0s - loss: 140.6090 - mean_squared_error: 140.6090
Epoch 00474: val_loss did not improve from 157.68224
526/526 [==============================] - 0s 459us/step - loss: 141.9344 - mean_squared_error: 141.9344 - val_loss: 168.1363 - val_mean_squared_error: 168.1363
Epoch 475/500
483/526 [==========================>...] - ETA: 0s - loss: 143.0884 - mean_squared_error: 143.0884
Epoch 00475: val_loss did not improve from 157.68224
526/526 [==============================] - 0s 462us/step - loss: 142.2213 - mean_squared_error: 142.2213 - val_loss: 161.9871 - val_mean_squared_error: 161.9871
Epoch 476/500
486/526 [==========================>...] - ETA: 0s - loss: 140.4365 - mean_squared_error: 140.4365
Epoch 00476: val_loss did not improve from 157.68224
526/526 [==============================] - 0s 459us/step - loss: 140.9655 - mean_squared_error: 140.9655 - val_loss: 160.3193 - val_mean_squared_error: 160.3193
Epoch 477/500
489/5

Epoch 499/500
479/526 [==========================>...] - ETA: 0s - loss: 136.2885 - mean_squared_error: 136.2885
Epoch 00499: val_loss did not improve from 157.46861
526/526 [==============================] - 0s 464us/step - loss: 136.8870 - mean_squared_error: 136.8870 - val_loss: 161.5036 - val_mean_squared_error: 161.5036
Epoch 500/500
486/526 [==========================>...] - ETA: 0s - loss: 138.6522 - mean_squared_error: 138.6522
Epoch 00500: val_loss improved from 157.46861 to 153.51118, saving model to model3.hdf5
526/526 [==============================] - 0s 609us/step - loss: 138.2877 - mean_squared_error: 138.2877 - val_loss: 153.5112 - val_mean_squared_error: 153.5112


In [15]:
model4 = models.Sequential()
model4.add(layers.Dense(256,activation="relu",kernel_regularizer=regularizers.l2(0.005),input_dim=81))
model4.add(layers.Dense(128,activation="relu",kernel_regularizer=regularizers.l2(0.005)))
model4.add(layers.Dense(128,activation="relu",kernel_regularizer=regularizers.l2(0.005)))
model4.add(layers.Dense(1))
model4.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss = losses.mean_squared_error,
    metrics = ['mean_squared_error']
)
file_path = "model4.hdf5"
checkpoint = ModelCheckpoint(file_path,monitor='val_loss', verbose=1,
                             save_best_only=True,period=1)
train_history4 = model4.fit(x_train,y_train,epochs=500,batch_size = 32,
                            validation_data = (x_val,y_val),callbacks = [checkpoint])

Epoch 1/500
515/526 [============================>.] - ETA: 0s - loss: 1298.5647 - mean_squared_error: 1296.7075
Epoch 00001: val_loss improved from inf to 936.60742, saving model to model4.hdf5
526/526 [==============================] - 0s 790us/step - loss: 1285.7001 - mean_squared_error: 1283.8464 - val_loss: 936.6074 - val_mean_squared_error: 934.9078
Epoch 2/500
513/526 [============================>.] - ETA: 0s - loss: 562.8146 - mean_squared_error: 561.1738
Epoch 00002: val_loss improved from 936.60742 to 601.59314, saving model to model4.hdf5
526/526 [==============================] - 0s 795us/step - loss: 563.0254 - mean_squared_error: 561.3857 - val_loss: 601.5931 - val_mean_squared_error: 599.9931
Epoch 3/500
519/526 [============================>.] - ETA: 0s - loss: 492.6719 - mean_squared_error: 491.0873
Epoch 00003: val_loss improved from 601.59314 to 374.95169, saving model to model4.hdf5
526/526 [==============================] - 0s 719us/step - loss: 492.3849 - mean_sq

516/526 [============================>.] - ETA: 0s - loss: 299.7035 - mean_squared_error: 298.1835
Epoch 00025: val_loss did not improve from 283.20569
526/526 [==============================] - 0s 535us/step - loss: 301.4528 - mean_squared_error: 299.9328 - val_loss: 363.1377 - val_mean_squared_error: 361.6191
Epoch 26/500
517/526 [============================>.] - ETA: 0s - loss: 304.9964 - mean_squared_error: 303.4792
Epoch 00026: val_loss did not improve from 283.20569
526/526 [==============================] - 0s 537us/step - loss: 304.7240 - mean_squared_error: 303.2068 - val_loss: 323.7941 - val_mean_squared_error: 322.2785
Epoch 27/500
513/526 [============================>.] - ETA: 0s - loss: 338.3136 - mean_squared_error: 336.7996
Epoch 00027: val_loss improved from 283.20569 to 264.74582, saving model to model4.hdf5
526/526 [==============================] - 0s 677us/step - loss: 336.4954 - mean_squared_error: 334.9814 - val_loss: 264.7458 - val_mean_squared_error: 263.2331


Epoch 50/500
520/526 [============================>.] - ETA: 0s - loss: 264.6659 - mean_squared_error: 263.2259
Epoch 00050: val_loss did not improve from 240.74149
526/526 [==============================] - 0s 533us/step - loss: 265.0235 - mean_squared_error: 263.5835 - val_loss: 259.2747 - val_mean_squared_error: 257.8361
Epoch 51/500
518/526 [============================>.] - ETA: 0s - loss: 253.5732 - mean_squared_error: 252.1354
Epoch 00051: val_loss did not improve from 240.74149
526/526 [==============================] - 0s 535us/step - loss: 253.2092 - mean_squared_error: 251.7715 - val_loss: 243.5749 - val_mean_squared_error: 242.1384
Epoch 52/500
515/526 [============================>.] - ETA: 0s - loss: 259.5090 - mean_squared_error: 258.0737
Epoch 00052: val_loss did not improve from 240.74149
526/526 [==============================] - 0s 537us/step - loss: 258.8655 - mean_squared_error: 257.4304 - val_loss: 363.7900 - val_mean_squared_error: 362.3557
Epoch 53/500
514/526 [

Epoch 75/500
516/526 [============================>.] - ETA: 0s - loss: 227.6302 - mean_squared_error: 226.1989
Epoch 00075: val_loss improved from 216.41327 to 212.61093, saving model to model4.hdf5
526/526 [==============================] - 0s 795us/step - loss: 227.7900 - mean_squared_error: 226.3587 - val_loss: 212.6109 - val_mean_squared_error: 211.1795
Epoch 76/500
517/526 [============================>.] - ETA: 0s - loss: 229.9531 - mean_squared_error: 228.5212
Epoch 00076: val_loss did not improve from 212.61093
526/526 [==============================] - 0s 535us/step - loss: 229.3193 - mean_squared_error: 227.8874 - val_loss: 232.4573 - val_mean_squared_error: 231.0250
Epoch 77/500
513/526 [============================>.] - ETA: 0s - loss: 231.1305 - mean_squared_error: 229.6976
Epoch 00077: val_loss did not improve from 212.61093
526/526 [==============================] - 0s 539us/step - loss: 230.2371 - mean_squared_error: 228.8042 - val_loss: 223.1498 - val_mean_squared_err

Epoch 100/500
518/526 [============================>.] - ETA: 0s - loss: 204.4797 - mean_squared_error: 203.0182
Epoch 00100: val_loss did not improve from 201.96393
526/526 [==============================] - 0s 533us/step - loss: 203.9351 - mean_squared_error: 202.4737 - val_loss: 227.3864 - val_mean_squared_error: 225.9241
Epoch 101/500
518/526 [============================>.] - ETA: 0s - loss: 205.2566 - mean_squared_error: 203.7929
Epoch 00101: val_loss did not improve from 201.96393
526/526 [==============================] - 0s 533us/step - loss: 205.0468 - mean_squared_error: 203.5832 - val_loss: 252.8015 - val_mean_squared_error: 251.3371
Epoch 102/500
516/526 [============================>.] - ETA: 0s - loss: 200.9245 - mean_squared_error: 199.4595
Epoch 00102: val_loss improved from 201.96393 to 197.47766, saving model to model4.hdf5
526/526 [==============================] - 0s 731us/step - loss: 200.6863 - mean_squared_error: 199.2213 - val_loss: 197.4777 - val_mean_squared_

...

Epoch 349/500
511/526 [============================>.] - ETA: 0s - loss: 111.4429 - mean_squared_error: 109.3784
Epoch 00349: val_loss did not improve from 127.87207
526/526 [==============================] - 0s 540us/step - loss: 112.1121 - mean_squared_error: 110.0475 - val_loss: 148.3236 - val_mean_squared_error: 146.2580
Epoch 350/500
512/526 [============================>.] - ETA: 0s - loss: 110.0362 - mean_squared_error: 107.9696
Epoch 00350: val_loss did not improve from 127.87207
526/526 [==============================] - 0s 540us/step - loss: 110.1997 - mean_squared_error: 108.1331 - val_loss: 145.0753 - val_mean_squared_error: 143.0076
Epoch 351/500
512/526 [============================>.] - ETA: 0s - loss: 109.2698 - mean_squared_error: 107.2011
Epoch 00351: val_loss did not improve from 127.87207
526/526 [==============================] - 0s 540us/step - loss: 109.1793 - mean_squared_error: 107.1105 - val_loss: 143.9791 - val_mean_squared_error: 141.9092
Epoch 352/500
509/5

511/526 [============================>.] - ETA: 0s - loss: 106.3571 - mean_squared_error: 104.1971
Epoch 00399: val_loss improved from 127.87207 to 127.77440, saving model to model4.hdf5
526/526 [==============================] - 0s 733us/step - loss: 105.9602 - mean_squared_error: 103.8002 - val_loss: 127.7744 - val_mean_squared_error: 125.6135
Epoch 400/500
516/526 [============================>.] - ETA: 0s - loss: 107.6757 - mean_squared_error: 105.5138
Epoch 00400: val_loss improved from 127.77440 to 126.44707, saving model to model4.hdf5
526/526 [==============================] - 0s 731us/step - loss: 107.2790 - mean_squared_error: 105.1171 - val_loss: 126.4471 - val_mean_squared_error: 124.2847
Epoch 401/500
518/526 [============================>.] - ETA: 0s - loss: 104.4509 - mean_squared_error: 102.2878
Epoch 00401: val_loss did not improve from 126.44707
526/526 [==============================] - 0s 533us/step - loss: 104.4551 - mean_squared_error: 102.2920 - val_loss: 134.380

Epoch 424/500
512/526 [============================>.] - ETA: 0s - loss: 108.2314 - mean_squared_error: 106.0309
Epoch 00424: val_loss did not improve from 126.44707
526/526 [==============================] - 0s 539us/step - loss: 107.9642 - mean_squared_error: 105.7637 - val_loss: 155.7003 - val_mean_squared_error: 153.4986
Epoch 425/500
516/526 [============================>.] - ETA: 0s - loss: 102.3708 - mean_squared_error: 100.1680
Epoch 00425: val_loss did not improve from 126.44707
526/526 [==============================] - 0s 537us/step - loss: 102.0863 - mean_squared_error: 99.8836 - val_loss: 139.8047 - val_mean_squared_error: 137.6012
Epoch 426/500
514/526 [============================>.] - ETA: 0s - loss: 104.8696 - mean_squared_error: 102.6655
Epoch 00426: val_loss did not improve from 126.44707
526/526 [==============================] - 0s 537us/step - loss: 104.4350 - mean_squared_error: 102.2309 - val_loss: 136.2667 - val_mean_squared_error: 134.0616
Epoch 427/500
511/52

Epoch 449/500
513/526 [============================>.] - ETA: 0s - loss: 100.4815 - mean_squared_error: 98.2374
Epoch 00449: val_loss did not improve from 124.96656
526/526 [==============================] - 0s 539us/step - loss: 100.3712 - mean_squared_error: 98.1271 - val_loss: 132.3496 - val_mean_squared_error: 130.1048
Epoch 450/500
514/526 [============================>.] - ETA: 0s - loss: 98.7406 - mean_squared_error: 96.4947
Epoch 00450: val_loss did not improve from 124.96656
526/526 [==============================] - 0s 537us/step - loss: 99.0538 - mean_squared_error: 96.8078 - val_loss: 135.4431 - val_mean_squared_error: 133.1955
Epoch 451/500
517/526 [============================>.] - ETA: 0s - loss: 102.8637 - mean_squared_error: 100.6152
Epoch 00451: val_loss did not improve from 124.96656
526/526 [==============================] - 0s 535us/step - loss: 102.7797 - mean_squared_error: 100.5312 - val_loss: 137.8927 - val_mean_squared_error: 135.6436
Epoch 452/500
515/526 [==

508/526 [===========================>..] - ETA: 0s - loss: 96.0426 - mean_squared_error: 93.7179
Epoch 00499: val_loss did not improve from 118.92934
526/526 [==============================] - 0s 542us/step - loss: 95.7961 - mean_squared_error: 93.4714 - val_loss: 144.9539 - val_mean_squared_error: 142.6286
Epoch 500/500
512/526 [============================>.] - ETA: 0s - loss: 95.3606 - mean_squared_error: 93.0349
Epoch 00500: val_loss did not improve from 118.92934
526/526 [==============================] - 0s 539us/step - loss: 95.2375 - mean_squared_error: 92.9118 - val_loss: 131.6003 - val_mean_squared_error: 129.2739


In [16]:
model5 = models.Sequential()
model5.add(layers.Dense(256,activation="relu",kernel_regularizer=regularizers.l1(0.005),input_dim=81))
model5.add(layers.Dense(128,activation="relu",kernel_regularizer=regularizers.l1(0.005)))
model5.add(layers.Dense(128,activation="relu",kernel_regularizer=regularizers.l1(0.005)))
model5.add(layers.Dense(1))
model5.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss = losses.mean_squared_error,
    metrics = ['mean_squared_error']
)
file_path = "model5.hdf5"
checkpoint = ModelCheckpoint(file_path,monitor='val_loss', verbose=1,
                             save_best_only=True,period=1)
train_history5 = model5.fit(x_train,y_train,epochs=500,batch_size = 32,
                            validation_data = (x_val,y_val),callbacks = [checkpoint])

Epoch 1/500
517/526 [============================>.] - ETA: 0s - loss: 4185.5264 - mean_squared_error: 4164.3105
Epoch 00001: val_loss improved from inf to 658.16937, saving model to model5.hdf5
526/526 [==============================] - 0s 933us/step - loss: 4127.1987 - mean_squared_error: 4106.0127 - val_loss: 658.1694 - val_mean_squared_error: 638.7418
Epoch 2/500
516/526 [============================>.] - ETA: 0s - loss: 568.0451 - mean_squared_error: 549.6063
Epoch 00002: val_loss improved from 658.16937 to 484.88010, saving model to model5.hdf5
526/526 [==============================] - 0s 784us/step - loss: 566.5416 - mean_squared_error: 548.1157 - val_loss: 484.8801 - val_mean_squared_error: 467.1358
Epoch 3/500
509/526 [============================>.] - ETA: 0s - loss: 477.1453 - mean_squared_error: 459.6022
Epoch 00003: val_loss improved from 484.88010 to 397.84381, saving model to model5.hdf5
526/526 [==============================] - 0s 752us/step - loss: 477.2976 - mean_sq

Epoch 25/500
509/526 [============================>.] - ETA: 0s - loss: 301.7913 - mean_squared_error: 286.1574
Epoch 00025: val_loss did not improve from 275.06793
526/526 [==============================] - 0s 542us/step - loss: 303.7250 - mean_squared_error: 288.0926 - val_loss: 277.4336 - val_mean_squared_error: 261.8473
Epoch 26/500
511/526 [============================>.] - ETA: 0s - loss: 307.4333 - mean_squared_error: 291.8926
Epoch 00026: val_loss did not improve from 275.06793
526/526 [==============================] - 0s 540us/step - loss: 306.9437 - mean_squared_error: 291.4044 - val_loss: 283.9955 - val_mean_squared_error: 268.4999
Epoch 27/500
516/526 [============================>.] - ETA: 0s - loss: 310.0744 - mean_squared_error: 294.6250
Epoch 00027: val_loss improved from 275.06793 to 274.85873, saving model to model5.hdf5
526/526 [==============================] - 0s 746us/step - loss: 311.5310 - mean_squared_error: 296.0825 - val_loss: 274.8587 - val_mean_squared_err

508/526 [===========================>..] - ETA: 0s - loss: 235.8539 - mean_squared_error: 223.8369
Epoch 00074: val_loss did not improve from 232.08781
526/526 [==============================] - 0s 542us/step - loss: 235.8884 - mean_squared_error: 223.8719 - val_loss: 241.3847 - val_mean_squared_error: 229.3818
Epoch 75/500
509/526 [============================>.] - ETA: 0s - loss: 230.4618 - mean_squared_error: 218.4735
Epoch 00075: val_loss improved from 232.08781 to 227.56624, saving model to model5.hdf5
526/526 [==============================] - 0s 767us/step - loss: 230.6526 - mean_squared_error: 218.6645 - val_loss: 227.5662 - val_mean_squared_error: 215.5851
Epoch 76/500
511/526 [============================>.] - ETA: 0s - loss: 229.5392 - mean_squared_error: 217.5784
Epoch 00076: val_loss improved from 227.56624 to 221.27219, saving model to model5.hdf5
526/526 [==============================] - 0s 727us/step - loss: 229.1596 - mean_squared_error: 217.1993 - val_loss: 221.2722 

Epoch 99/500
515/526 [============================>.] - ETA: 0s - loss: 215.1595 - mean_squared_error: 203.7933
Epoch 00099: val_loss did not improve from 210.07965
526/526 [==============================] - 0s 539us/step - loss: 216.5283 - mean_squared_error: 205.1624 - val_loss: 261.4953 - val_mean_squared_error: 250.1373
Epoch 100/500
515/526 [============================>.] - ETA: 0s - loss: 217.2457 - mean_squared_error: 205.8943
Epoch 00100: val_loss did not improve from 210.07965
526/526 [==============================] - 0s 535us/step - loss: 217.1622 - mean_squared_error: 205.8110 - val_loss: 222.3927 - val_mean_squared_error: 211.0475
Epoch 101/500
511/526 [============================>.] - ETA: 0s - loss: 218.0630 - mean_squared_error: 206.7201
Epoch 00101: val_loss did not improve from 210.07965
526/526 [==============================] - 0s 539us/step - loss: 217.8800 - mean_squared_error: 206.5369 - val_loss: 219.6397 - val_mean_squared_error: 208.2928
Epoch 102/500
511/52

Epoch 124/500
512/526 [============================>.] - ETA: 0s - loss: 202.9253 - mean_squared_error: 191.8572
Epoch 00124: val_loss did not improve from 198.98285
526/526 [==============================] - 0s 540us/step - loss: 201.5317 - mean_squared_error: 190.4637 - val_loss: 203.0224 - val_mean_squared_error: 191.9593
Epoch 125/500
511/526 [============================>.] - ETA: 0s - loss: 200.4429 - mean_squared_error: 189.3902
Epoch 00125: val_loss did not improve from 198.98285
526/526 [==============================] - 0s 539us/step - loss: 199.8878 - mean_squared_error: 188.8354 - val_loss: 254.9596 - val_mean_squared_error: 243.9159
Epoch 126/500
513/526 [============================>.] - ETA: 0s - loss: 197.5199 - mean_squared_error: 186.4729
Epoch 00126: val_loss did not improve from 198.98285
526/526 [==============================] - 0s 539us/step - loss: 197.5835 - mean_squared_error: 186.5366 - val_loss: 220.5798 - val_mean_squared_error: 209.5359
Epoch 127/500
513/5

...

Epoch 349/500
514/526 [============================>.] - ETA: 0s - loss: 126.6578 - mean_squared_error: 115.4618
Epoch 00349: val_loss did not improve from 142.20178
526/526 [==============================] - 0s 537us/step - loss: 126.6681 - mean_squared_error: 115.4722 - val_loss: 147.6761 - val_mean_squared_error: 136.4845
Epoch 350/500
513/526 [============================>.] - ETA: 0s - loss: 126.1967 - mean_squared_error: 114.9964
Epoch 00350: val_loss did not improve from 142.20178
526/526 [==============================] - 0s 537us/step - loss: 126.7366 - mean_squared_error: 115.5363 - val_loss: 148.3482 - val_mean_squared_error: 137.1470
Epoch 351/500
510/526 [============================>.] - ETA: 0s - loss: 124.7750 - mean_squared_error: 113.5648
Epoch 00351: val_loss did not improve from 142.20178
526/526 [==============================] - 0s 540us/step - loss: 124.6851 - mean_squared_error: 113.4744 - val_loss: 166.1295 - val_mean_squared_error: 154.9074
Epoch 352/500
506/5

Epoch 399/500
491/526 [===========================>..] - ETA: 0s - loss: 121.0026 - mean_squared_error: 109.7336
Epoch 00399: val_loss did not improve from 139.80577
526/526 [==============================] - 0s 561us/step - loss: 121.0794 - mean_squared_error: 109.8111 - val_loss: 145.6537 - val_mean_squared_error: 134.3951
Epoch 400/500
493/526 [===========================>..] - ETA: 0s - loss: 121.1186 - mean_squared_error: 109.8559
Epoch 00400: val_loss did not improve from 139.80577
526/526 [==============================] - 0s 558us/step - loss: 121.0302 - mean_squared_error: 109.7676 - val_loss: 142.2043 - val_mean_squared_error: 130.9422
Epoch 401/500
486/526 [==========================>...] - ETA: 0s - loss: 122.2083 - mean_squared_error: 110.9471
Epoch 00401: val_loss did not improve from 139.80577
526/526 [==============================] - 0s 565us/step - loss: 121.7360 - mean_squared_error: 110.4745 - val_loss: 153.5177 - val_mean_squared_error: 142.2535
Epoch 402/500
488/5

Epoch 424/500
487/526 [==========================>...] - ETA: 0s - loss: 118.7323 - mean_squared_error: 107.4430
Epoch 00424: val_loss did not improve from 138.38559
526/526 [==============================] - 0s 565us/step - loss: 118.7502 - mean_squared_error: 107.4609 - val_loss: 158.6399 - val_mean_squared_error: 147.3482
Epoch 425/500
486/526 [==========================>...] - ETA: 0s - loss: 121.5162 - mean_squared_error: 110.2142
Epoch 00425: val_loss did not improve from 138.38559
526/526 [==============================] - 0s 567us/step - loss: 120.9847 - mean_squared_error: 109.6831 - val_loss: 145.9702 - val_mean_squared_error: 134.6748
Epoch 426/500
489/526 [==========================>...] - ETA: 0s - loss: 119.1537 - mean_squared_error: 107.8457
Epoch 00426: val_loss did not improve from 138.38559
526/526 [==============================] - 0s 565us/step - loss: 118.3127 - mean_squared_error: 107.0047 - val_loss: 141.7712 - val_mean_squared_error: 130.4615
Epoch 427/500
492/5

Epoch 474/500
514/526 [============================>.] - ETA: 0s - loss: 113.6771 - mean_squared_error: 102.3534
Epoch 00474: val_loss did not improve from 137.46976
526/526 [==============================] - 0s 537us/step - loss: 113.3907 - mean_squared_error: 102.0672 - val_loss: 144.1906 - val_mean_squared_error: 132.8740
Epoch 475/500
510/526 [============================>.] - ETA: 0s - loss: 118.6763 - mean_squared_error: 107.3501
Epoch 00475: val_loss did not improve from 137.46976
526/526 [==============================] - 0s 542us/step - loss: 118.1508 - mean_squared_error: 106.8243 - val_loss: 149.3781 - val_mean_squared_error: 138.0420
Epoch 476/500
512/526 [============================>.] - ETA: 0s - loss: 112.9122 - mean_squared_error: 101.5746
Epoch 00476: val_loss did not improve from 137.46976
526/526 [==============================] - 0s 539us/step - loss: 113.2532 - mean_squared_error: 101.9155 - val_loss: 139.7113 - val_mean_squared_error: 128.3677
Epoch 477/500
511/5

Epoch 499/500
516/526 [============================>.] - ETA: 0s - loss: 111.1964 - mean_squared_error: 99.8385
Epoch 00499: val_loss did not improve from 135.59554
526/526 [==============================] - 0s 537us/step - loss: 110.7738 - mean_squared_error: 99.4161 - val_loss: 149.1766 - val_mean_squared_error: 137.8263
Epoch 500/500
511/526 [============================>.] - ETA: 0s - loss: 111.1161 - mean_squared_error: 99.7630 
Epoch 00500: val_loss did not improve from 135.59554
526/526 [==============================] - 0s 542us/step - loss: 111.3567 - mean_squared_error: 100.0035 - val_loss: 140.1854 - val_mean_squared_error: 128.8290


In [17]:
model6 = models.Sequential()
model6.add(layers.Dense(256,activation="relu",input_dim=81))
model6.add(layers.Dropout(0.3))
model6.add(layers.Dense(128,activation="relu"))
model6.add(layers.Dropout(0.3))
model6.add(layers.Dense(128,activation="relu"))
model6.add(layers.Dense(1))
model6.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss = losses.mean_squared_error,
    metrics = ['mean_squared_error']
)
file_path = "model6.hdf5"
checkpoint = ModelCheckpoint(file_path,monitor='val_loss', verbose=1,
                             save_best_only=True,period=1)
train_history6 = model6.fit(x_train,y_train,epochs=500,batch_size = 32,
                            validation_data = (x_val,y_val),callbacks = [checkpoint])

Epoch 1/500
473/526 [=========================>....] - ETA: 0s - loss: 144613.0781 - mean_squared_error: 144613.0781
Epoch 00001: val_loss improved from inf to 2953.68652, saving model to model6.hdf5
526/526 [==============================] - 1s 1ms/step - loss: 133634.4844 - mean_squared_error: 133634.4844 - val_loss: 2953.6865 - val_mean_squared_error: 2953.6865
Epoch 2/500
471/526 [=========================>....] - ETA: 0s - loss: 22760.4805 - mean_squared_error: 22760.4805
Epoch 00002: val_loss did not improve from 2953.68652
526/526 [==============================] - 0s 792us/step - loss: 22159.4180 - mean_squared_error: 22159.4180 - val_loss: 3501.6038 - val_mean_squared_error: 3501.6038
Epoch 3/500
476/526 [==========================>...] - ETA: 0s - loss: 11480.0039 - mean_squared_error: 11480.0039
Epoch 00003: val_loss did not improve from 2953.68652
526/526 [==============================] - 0s 784us/step - loss: 11300.2314 - mean_squared_error: 11300.2314 - val_loss: 3175.97

Epoch 49/500
474/526 [==========================>...] - ETA: 0s - loss: 303.7328 - mean_squared_error: 303.7328
Epoch 00049: val_loss did not improve from 439.92395
526/526 [==============================] - 0s 788us/step - loss: 305.0126 - mean_squared_error: 305.0126 - val_loss: 630.1425 - val_mean_squared_error: 630.1425
Epoch 50/500
472/526 [=========================>....] - ETA: 0s - loss: 303.2968 - mean_squared_error: 303.2968
Epoch 00050: val_loss did not improve from 439.92395
526/526 [==============================] - 0s 790us/step - loss: 304.9359 - mean_squared_error: 304.9359 - val_loss: 677.1832 - val_mean_squared_error: 677.1832
Epoch 51/500
469/526 [=========================>....] - ETA: 0s - loss: 302.3285 - mean_squared_error: 302.3285
Epoch 00051: val_loss did not improve from 439.92395
526/526 [==============================] - 0s 793us/step - loss: 299.0535 - mean_squared_error: 299.0535 - val_loss: 527.0965 - val_mean_squared_error: 527.0965
Epoch 52/500
477/526 [

462/526 [=========================>....] - ETA: 0s - loss: 274.7199 - mean_squared_error: 274.7199
Epoch 00074: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 811us/step - loss: 273.2762 - mean_squared_error: 273.2762 - val_loss: 591.6721 - val_mean_squared_error: 591.6721
Epoch 75/500
523/526 [============================>.] - ETA: 0s - loss: 274.0916 - mean_squared_error: 274.0916
Epoch 00075: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 816us/step - loss: 274.6663 - mean_squared_error: 274.6663 - val_loss: 598.4849 - val_mean_squared_error: 598.4849
Epoch 76/500
471/526 [=========================>....] - ETA: 0s - loss: 271.1904 - mean_squared_error: 271.1904
Epoch 00076: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 793us/step - loss: 270.9664 - mean_squared_error: 270.9664 - val_loss: 584.6513 - val_mean_squared_error: 584.6513
Epoch 77/500
521/526 [=============

Epoch 124/500
509/526 [============================>.] - ETA: 0s - loss: 246.8214 - mean_squared_error: 246.8214
Epoch 00124: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 841us/step - loss: 246.9738 - mean_squared_error: 246.9738 - val_loss: 554.6473 - val_mean_squared_error: 554.6473
Epoch 125/500
509/526 [============================>.] - ETA: 0s - loss: 245.0353 - mean_squared_error: 245.0353
Epoch 00125: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 843us/step - loss: 245.3835 - mean_squared_error: 245.3835 - val_loss: 645.2305 - val_mean_squared_error: 645.2305
Epoch 126/500
508/526 [===========================>..] - ETA: 0s - loss: 244.4647 - mean_squared_error: 244.4647
Epoch 00126: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 843us/step - loss: 245.2151 - mean_squared_error: 245.2151 - val_loss: 561.0667 - val_mean_squared_error: 561.0667
Epoch 127/500
505/5

461/526 [=========================>....] - ETA: 0s - loss: 234.5370 - mean_squared_error: 234.5370
Epoch 00174: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 908us/step - loss: 231.9785 - mean_squared_error: 231.9785 - val_loss: 585.1898 - val_mean_squared_error: 585.1898
Epoch 175/500
461/526 [=========================>....] - ETA: 0s - loss: 233.7340 - mean_squared_error: 233.7340
Epoch 00175: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 811us/step - loss: 234.7274 - mean_squared_error: 234.7274 - val_loss: 617.1290 - val_mean_squared_error: 617.1290
Epoch 176/500
521/526 [============================>.] - ETA: 0s - loss: 232.3424 - mean_squared_error: 232.3424
Epoch 00176: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 820us/step - loss: 232.1721 - mean_squared_error: 232.1721 - val_loss: 606.2203 - val_mean_squared_error: 606.2203
Epoch 177/500
463/526 [==========

...

Epoch 299/500
526/526 [==============================] - ETA: 0s - loss: 211.6997 - mean_squared_error: 211.6997
Epoch 00299: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 816us/step - loss: 211.6997 - mean_squared_error: 211.6997 - val_loss: 551.4250 - val_mean_squared_error: 551.4250
Epoch 300/500
524/526 [============================>.] - ETA: 0s - loss: 210.5408 - mean_squared_error: 210.5408
Epoch 00300: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 819us/step - loss: 210.5092 - mean_squared_error: 210.5092 - val_loss: 554.2047 - val_mean_squared_error: 554.2047
Epoch 301/500
461/526 [=========================>....] - ETA: 0s - loss: 214.0078 - mean_squared_error: 214.0078
Epoch 00301: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 814us/step - loss: 213.0963 - mean_squared_error: 213.0963 - val_loss: 536.0894 - val_mean_squared_error: 536.0894
Epoch 302/500
525/5

512/526 [============================>.] - ETA: 0s - loss: 205.5930 - mean_squared_error: 205.5930
Epoch 00349: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 838us/step - loss: 205.7522 - mean_squared_error: 205.7522 - val_loss: 524.7547 - val_mean_squared_error: 524.7547
Epoch 350/500
498/526 [===========================>..] - ETA: 0s - loss: 201.4241 - mean_squared_error: 201.4241
Epoch 00350: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 854us/step - loss: 202.0102 - mean_squared_error: 202.0102 - val_loss: 524.9559 - val_mean_squared_error: 524.9559
Epoch 351/500
511/526 [============================>.] - ETA: 0s - loss: 206.5679 - mean_squared_error: 206.5679
Epoch 00351: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 837us/step - loss: 206.5264 - mean_squared_error: 206.5264 - val_loss: 524.6537 - val_mean_squared_error: 524.6537
Epoch 352/500
513/526 [==========

522/526 [============================>.] - ETA: 0s - loss: 198.7142 - mean_squared_error: 198.7142
Epoch 00399: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 818us/step - loss: 198.2398 - mean_squared_error: 198.2398 - val_loss: 506.6788 - val_mean_squared_error: 506.6788
Epoch 400/500
516/526 [============================>.] - ETA: 0s - loss: 199.0380 - mean_squared_error: 199.0380
Epoch 00400: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 830us/step - loss: 199.0395 - mean_squared_error: 199.0395 - val_loss: 492.1376 - val_mean_squared_error: 492.1376
Epoch 401/500
522/526 [============================>.] - ETA: 0s - loss: 199.2658 - mean_squared_error: 199.2658
Epoch 00401: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 818us/step - loss: 199.1342 - mean_squared_error: 199.1342 - val_loss: 530.0208 - val_mean_squared_error: 530.0208
Epoch 402/500
519/526 [==========

521/526 [============================>.] - ETA: 0s - loss: 192.5316 - mean_squared_error: 192.5316
Epoch 00449: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 822us/step - loss: 192.5150 - mean_squared_error: 192.5150 - val_loss: 551.8720 - val_mean_squared_error: 551.8720
Epoch 450/500
525/526 [============================>.] - ETA: 0s - loss: 193.3547 - mean_squared_error: 193.3547
Epoch 00450: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 816us/step - loss: 193.4746 - mean_squared_error: 193.4746 - val_loss: 522.2521 - val_mean_squared_error: 522.2521
Epoch 451/500
525/526 [============================>.] - ETA: 0s - loss: 194.4821 - mean_squared_error: 194.4821
Epoch 00451: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 813us/step - loss: 194.4127 - mean_squared_error: 194.4127 - val_loss: 602.7739 - val_mean_squared_error: 602.7739
Epoch 452/500
523/526 [==========

525/526 [============================>.] - ETA: 0s - loss: 190.4272 - mean_squared_error: 190.4272
Epoch 00499: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 814us/step - loss: 190.4469 - mean_squared_error: 190.4469 - val_loss: 511.3651 - val_mean_squared_error: 511.3651
Epoch 500/500
526/526 [==============================] - ETA: 0s - loss: 190.9642 - mean_squared_error: 190.9642
Epoch 00500: val_loss did not improve from 413.63632
526/526 [==============================] - 0s 813us/step - loss: 190.9642 - mean_squared_error: 190.9642 - val_loss: 510.3230 - val_mean_squared_error: 510.3230


In [18]:
print("model1 best val mse:{}".format(min(train_history1.history['val_loss'])))
print("model2 best val mse:{}".format(min(train_history2.history['val_loss'])))
print("model3 best val mse:{}".format(min(train_history3.history['val_loss'])))
print("model4 best val mse:{}".format(min(train_history4.history['val_loss'])))
print("model5 best val mse:{}".format(min(train_history5.history['val_loss'])))
print("model6 best val mse:{}".format(min(train_history6.history['val_loss'])))

model1 best val mse:123.40611267089844
model2 best val mse:211.79542541503906
model3 best val mse:153.5111846923828
model4 best val mse:118.9293441772461
model5 best val mse:135.5955352783203
model6 best val mse:413.6363220214844


#### Part b
As we can see, the best model here is model4 which is 3 hidden layers with l2 norm.
We load the best model, predict for test set and get the test mse is 
136.88799634697085.

In [19]:
best_model = models.load_model("model4.hdf5")
y_hat_test = best_model.predict(x_test)
print(mean_squared_error(y_hat_test,y_test))

136.88799634697085


### Question 2
We will upload the script we run in cluster,
we will also run that code here with local pyspark
(but no hadoop, and a little bit change in the code) here.

In [2]:
spark_home = "F:\spark-3.1.2-bin-hadoop3.2"
python_path = "E:\python\python.exe"
findspark.init(spark_home,python_path)

In [3]:
user_key = pd.read_csv("stats507/user_key.csv")
tran = pd.read_csv("stats507/triangles.csv")
rect = pd.read_csv("stats507/rectangles.csv")

In [4]:
spark = SparkSession \
    .builder \
    .appName('ps8_qs2') \
    .getOrCreate()
tran_res = {}
rect_res = {}

In [5]:
user_key = spark.createDataFrame(user_key)
user_key = user_key.filter(user_key.user == "tiejin")
tran = spark.createDataFrame(tran)
rect = spark.createDataFrame(rect)

In [6]:
my_tran = user_key.join(tran,user_key.key==tran.key,'left')
my_rect = user_key.join(rect,user_key.key==rect.key,'left')

In [7]:
my_tran_area =my_tran.withColumn("area",my_tran['base']*my_tran['height']*0.5)
my_rect_area =my_rect.withColumn("area",my_rect['width']*my_rect['length'])

In [8]:
mean_tran = my_tran_area.select(mean('area')).first()[0]
mean_rect = my_rect_area.select(mean('area')).first()[0]

In [9]:
total_tran = my_tran_area.select('area').rdd.map(lambda x:x[0]).reduce(lambda x,y:x+y)
total_rect = my_rect_area.select('area').rdd.map(lambda x:x[0]).reduce(lambda x,y:x+y)

In [10]:
tran_res['number'] = my_tran_area.select('area').rdd.count()
tran_res['total areas'] = total_tran
tran_res['mean areas'] = mean_tran
rect_res['number'] = my_rect_area.select('area').rdd.count()
rect_res['total areas'] = total_rect
rect_res['mean areas'] = mean_rect

final_dict = {"triangles":tran_res,
              "rectangles":rect_res}

In [11]:
final_df = pd.DataFrame(final_dict)
final_df.to_csv("ps8_q2_tiejin_results.csv")

After all the code finish, we can get the csv file, and we read it here.

In [12]:
df = pd.read_csv("ps8_q2_tiejin_results.csv",index_col=0).T

In [13]:
df['number'] = df['number'].astype("int")
df['total areas'] = df['total areas'].apply(lambda x:round(x,2))

In [14]:
df

,number,total areas,mean areas
triangles,5937,23707233.14,3993.133424
rectangles,6087,12590875.40,2068.486183
